In [1]:
import omnixas
from omnixas.data.xas import ElementSpectrum, IntensityValues, Material, EnergyGrid
from omnixas.core.periodic_table import Element, SpectrumType
from omnixas.featurizer.m3gnet_featurizer import M3GNetSiteFeaturizer, M3GNetFeaturizer
import numpy as np
import os

MP_API_KEY='ELN6GqAzO6OzNDHgmxecYtZ5cDsY16ti'

from mp_api.client import MPRester
from emmet.core.xas import Edge, XASDoc, Type
from pymatgen.core.structure import Structure

import umap.umap_ as umap

In [4]:
os.listdir('omnixas/material_id_and_site')

['Fe_FEFF_train.txt',
 'Mn_FEFF_val.txt',
 'Cu_VASP_test.txt',
 'Ni_FEFF_val.txt',
 'Mn_FEFF_test.txt',
 'Cr_FEFF_train.txt',
 'Cr_FEFF_test.txt',
 'Co_FEFF_val.txt',
 'Ni_FEFF_test.txt',
 'Cu_FEFF_train.txt',
 'Cu_VASP_train.txt',
 'V_FEFF_test.txt',
 'sanitycheck2.ipynb',
 'Ti_FEFF_train.txt',
 'Ti_VASP_train.txt',
 'Ni_FEFF_train.txt',
 'Cu_FEFF_test.txt',
 'Co_FEFF_train.txt',
 'Ti_VASP_val.txt',
 'Ti_FEFF_test.txt',
 'Cu_VASP_val.txt',
 'Ti_FEFF_val.txt',
 'Cr_FEFF_val.txt',
 'Cu_FEFF_val.txt',
 'V_FEFF_val.txt',
 'Mn_FEFF_train.txt',
 'Ti_VASP_test.txt',
 'V_FEFF_train.txt',
 'Fe_FEFF_val.txt',
 'Fe_FEFF_test.txt',
 'Co_FEFF_test.txt']

In [5]:
big_lst = []
elem = []
for f in os.listdir('omnixas/material_id_and_site'):
    if f == 'sanitycheck2.ipynb':
        continue
    with open("omnixas/material_id_and_site/"+f, 'r') as file:
        lines = file.readlines()
                # Remove newline characters from each line
        lst = [line.strip() for line in lines]
        big_lst.extend(lst)

        for l in lst:
            elem.append(f[:2])

In [9]:
matids = []
for i in range(len(big_lst)): 
    matids.append(big_lst[i][:-4]) 

In [10]:
big_spc = []

for f in os.listdir('omnixas/ml_data'):
    if f[-5] != 'y':
        continue
    with open("omnixas/ml_data/"+f, 'r') as file:
        lines = file.readlines()
                # Remove newline characters from each line
        spc = [line.strip() for line in lines]
        big_spc.extend(spc)

new_spc = []
for i in range(len(big_spc)):
    float_list = list(map(float, big_spc[i].split()))
    new_spc.append(float_list)

In [24]:
features = []
spectra = []

elements = list(set(elem))
elements[3] = elements[3][0]
fields = ["material_id",
          'structure']
structs = {}

for el in elements:
    with MPRester(MP_API_KEY) as mpr:
        docs = mpr.materials.summary.search(elements=[el],fields=fields)

    structs.update({d.material_id: d.structure for d in docs})

kys = list(structs.keys())

for id in kys:
    if id not in matids:
        del structs[id]

for i in range(len(big_lst)):

    ID = int(big_lst[i][-3:])
    mat = big_lst[i][:-4]
    
    if mat not in structs.keys():
        continue
    struct = structs[mat]
    if struct == []:
        continue
    # if FEFF_IDs[i] not in FEFF_structures:
    #     continue

    # if str(FEFF_structures[FEFF_IDs[i]].sites[int(FEFF_sites[i])].species)[:2] != "Cu":
    #     continue

    spectrum = ElementSpectrum(
        element=Element.Cu,
        type=SpectrumType.FEFF,
        index=ID,
        material= Material( id = mat, structure = struct),
        intensities=IntensityValues(new_spc[i]),
        energies=EnergyGrid(root=FEFF_energies),
    )

    # featurize material structure corresponding to the spectrum_1
    feature = M3GNetSiteFeaturizer().featurize(spectrum.material.structure, spectrum.index)

    features.append(feature)
    spectra.append(spectrum)

Retrieving SummaryDoc documents:   0%|          | 0/6846 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/9871 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/14071 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/9791 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/7727 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/12967 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/11106 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/8305 [00:00<?, ?it/s]

NameError: name 'FEFF_energies' is not defined

In [ ]:
np.random.seed(208374823)

#idx = np.random.choice(range(len(features)),200000)
subset = np.array(features)
reducer = umap.UMAP(random_state=12934, n_neighbors = 25,verbose = True, n_jobs = 6).fit(subset)#,n_neighbors=, min_dist=)
embedding = reducer.transform(subset)

Retrieving SummaryDoc documents:   0%|          | 0/298 [00:00<?, ?it/s]

In [ ]:
embedding_x = embedding[:,0]
embedding_y = embedding[:,1]

fig, ax = plt.subplots(figsize = (15,10))

scatter = ax.scatter(embedding_x, embedding_y, s=3)

['Cr', 'Cu', 'Mn', 'V', 'Ti', 'Fe', 'Co', 'Ni']